In [1]:
import numpy as np

In [2]:
def sol_trinffil(A, b):    
    #assert np.prod(np.diag(A)) != 0,"Error: la matriz es singular"
    
    n = len(b)
    x = b.copy()

    if n==1:
        x[0]=x[0]/A[0, 0]
    else:
        k=0
        while x[k]==0 and k<n-1: #Si b_i = 0 => x_i = 0
            k+=1
    
        for i in range(k,n): #Ya calculé los x hasta k
            x[i] = (x[i]-A[i,0:i]@x[0:i]) / A[i,i]
    
    return x

def sol_trsupfil(A, b):    
    #assert np.prod(np.diag(A)) != 0,"Error: la matriz es singular"
    n = len(b)-1
    x = b.copy() 
    k=n
    while x[k]==0 and k>0: #Si b_i = 0 => x_i = 0
        k-=1

    for i in reversed(range(k+1)): #Ya calculé los x desde k hasta n.
        x[i] = (x[i]-(A[i,i+1:n+1]@x[i+1:n+1])) / A[i,i]
    return x

def sol_defpos(B,y):
    A=B.copy()
    b=y.copy()
    G=np.linalg.cholesky(A).T
    y = sol_trinffil(G.T,b)
    x=sol_trsupfil(G, y)
    return x

In [3]:
def self_adjoint_bvp(fun, p, q, a, b, ua, s, t, d, n):
    u=np.zeros(n+1) #solutions
    u[0]=ua
    
    #discretiazacion
    h=(b-a)/n
    steps_full=np.linspace(a,b,n+1)
    steps_half=np.delete(steps_full + h/2,n)
    
    
    #definimos A
    A=np.zeros((n-1,n-1))
    #diagonal
    p1=p(steps_half[0])
    for i in range(n-2):
        p2=p(steps_half[i+1])
        A[i,i]= -((p1+p2)/(h**2)) - q(steps_full[i+1])
        p1=p2

    A[n-2, n-2]= -((p(steps_half[-1]) + p(steps_half[-2]))/(h**2))-q(steps_full[-2]) + ((t/((s*h)+t))*(p(steps_half[-1])/(h**2)))
    #diagonal inf y sup
    for i in range(n-2):
        p1=p(steps_half[i+1])
        A[i,i+1]=p1/(h**2) #Sup
        A[i+1,i]=A[i,i+1]
    
    
    #definimos f
    f=fun(steps_full[1:-1])
    f[0]=f[0]-(p(steps_half[0])*ua/(h**2))
    f[-1]=f[-1]-(d*(h/((s*h)+t))*(p(steps_half[-1])/(h**2)))
    
    
    #Resolvemos Au[1:-1]=f
    u[1:-1]=sol_defpos(-A,-f) #A is def neg => -A is def pos
    u[-1]=(d + ((t/h)*u[-2]))/(s+(t/h))
    return u,steps_full